In [3]:
import numpy as np
from sklearn import tree
import pandas as pd
import random
from __future__ import division
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import paired_distances as dist
import matplotlib.pyplot as plt

#opening the file in read mode
df = pd.read_csv('..\week5\SFPD_Incidents_-_from_1_January_2003.csv')

#In this exercise we explore KK-means clustering
#and we it out on the locations of the PROSTITUTION crime type.
locations_data = df[df['Category'] == 'PROSTITUTION'].loc[ : , ['X','Y']].astype(float)

locations_data = locations_data[locations_data.X < -122] #DELETING data errors inclusions

squared_errors = []
number_of_clusters_list = range(2,7)
clusters_predictions = []
points_data = locations_data[['X','Y']]
cluster_centers_data = pd.DataFrame(columns=['X', 'Y', 'num_K','clustering_index', 'avg_sq_Err'])

for number_of_clusters in number_of_clusters_list:
    
    KM = KMeans(n_clusters = number_of_clusters).fit(points_data)
    #saving the cluster centers to calculate the squared errors later
    cluster_centers = KM.cluster_centers_
    clusters_predictions = KM.predict(points_data)
    #insert predictions as column in the dataset
    locations_data.insert(number_of_clusters,'K'+str(number_of_clusters),clusters_predictions)#(loc, column, value,)
    #list of all the centers in relative position to all the data
    cluster_center_of_points = map(lambda(index): cluster_centers[index],clusters_predictions)
    #the squared error is the sum of the squared distances between each point and the relative cluster center
    squared_errors.append(sum(dist(cluster_center_of_points, points_data.values, squared=True)))
    #adding cluster centers info to the dataset
    for index in range(len(cluster_centers)):
        cluster_centers_data.loc[len(cluster_centers_data)] = [cluster_centers[index][0],cluster_centers[index][1],number_of_clusters,index,squared_errors[-1]]

print 'Squared error differences between different values of K :'
for i in range(len(squared_errors)-1):
    print '{:2.3f} error difference with '.format(squared_errors[i] - squared_errors[i+1]),(i+2),' and ',(i+3),' clusters.'

#print locations_data.head()
#print cluster_centers_data
#Saving data to be displayed in D3
locations_data.to_json('prost_locations.json',orient='values')
cluster_centers_data.to_json('cluster_centers_locations.json',orient='values')



plt.figure("Squared error with number of Clusters")    
plt.plot(number_of_clusters_list, squared_errors)
plt.xticks(number_of_clusters_list)
plt.xlabel("k")
plt.ylabel("total squared error")
plt.title("Total Error vs. # of Clusters")
#IF YOU WANT THE PLOT UNCHECK THIS (IT WILL BLOCK PART OF PREVIOUS CODE)
#plt.show()


IOError: File ..\week5\SFPD_Incidents_-_from_1_January_2003.csv does not exist